## Time to apply to Video! </br>
the next step would be to analyze these for the whole video. </br>
So keep the background image, load every new frame of the video, run your calculations, and then save it to a video file. This tutorial might be helpful.

In [1]:
# example codes
# reference : https://learnopencv.com/reading-and-writing-videos-using-opencv/
import cv2 
 
# Create a video capture object, in this case we are reading the video from a file
vid_capture = cv2.VideoCapture('Resources/Cars.mp4')
 
if (vid_capture.isOpened() == False):
  print("Error opening the video file")
# Read fps and frame count
else:
  # Get frame rate information
  # You can replace 5 with CAP_PROP_FPS as well, they are enumerations
  fps = vid_capture.get(5)
  print('Frames per second : ', fps,'FPS')
 
  # Get frame count
  # You can replace 7 with CAP_PROP_FRAME_COUNT as well, they are enumerations
  frame_count = vid_capture.get(7)
  print('Frame count : ', frame_count)
 
while(vid_capture.isOpened()):
  # vid_capture.read() methods returns a tuple, first element is a bool 
  # and the second is frame
  ret, frame = vid_capture.read()
  if ret == True:
    cv2.imshow('Frame',frame)
    # 20 is in milliseconds, try to increase the value, say 50 and observe
    key = cv2.waitKey(20)
     
    if key == ord('q'):
      break
  else:
    break
 
# Release the video capture object
vid_capture.release()
cv2.destroyAllWindows()

Error opening the video file


OpenCV: Couldn't read video stream from file "Resources/Cars.mp4"


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

color_directory = "2018-04-20-10-04-11/kinect/color"
depth_directory = "2018-04-20-10-04-11/kinect/depth"

color_filenames = os.listdir(color_directory)
depth_filenames = os.listdir(depth_directory)

color_filenames.sort()
depth_filenames.sort()

#background_depth = cv2.imread(os.path.join(depth_directory, depth_filenames[0]))
background_depth= np.load(os.path.join(depth_directory, depth_filenames[0]))

output_filename = 'output_video.mov'
fourcc = cv2.VideoWriter_fourcc(*'h264') # 비디오 코덱을 지정하는 4자 코드
fps = 25.0 # 초당 프레임 수
frame_size = (960,540) # 프레임 크기 (예: 가로 640, 세로 480)
result_vid = cv2.VideoWriter(output_filename, fourcc, fps, frame_size) # VideoWriter 객체 생성
# with color_filenames, convert to frames and store as video 
for filename in color_filenames:
    frame = cv2.imread(os.path.join(color_directory, filename))  # 파일을 읽어 프레임으로 변환
    result_vid.write(frame)  # 프레임을 결과 비디오에 추가


#result_vid.write(color_filenames)
result_vid.release() 

prev_bounding_box = ...

for filename in depth_filenames:

    # load Image
    depth_image = np.load(os.path.join(depth_directory, filename)) # 파일을 읽어옴
    color_filename = filename.split(".")[0] + ".png"
    color_filename = os.path.join(color_directory,color_filename)
    
    if not os.path.exists(color_filename):
        print("file not found")
        continue
    
    color_image = cv2.imread(color_filename)
    
    # take depth difference
    diff = depth_image-background_depth

    ## process bounding box - copied from 'practice1.ipynb'
    # Normalize the difference image to range [0, 255]
    normalized_diff = cv2.normalize(np.abs(diff), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # Binarize
    _, im_th = cv2.threshold(normalized_diff, 30, 255, cv2.THRESH_BINARY)
    # Taking a matrix of size 5 as the kernel ; gernerate 5*5 array filled with 1
    kernel = np.ones((5, 5), np.uint8) 

    img_erosion = cv2.erode(im_th, kernel, iterations=3) 
    img_dilation = cv2.dilate(im_th, kernel, iterations=3) 
    # Get all group of pixels using cv.connectedComponentsWithStats()
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(img_erosion)

    #print(np.amin(depth_image),np.amax(depth_image))
    #depth_image = cv2.cvtColor(depth_image,cv2.COLOR_GRAY2BGR)

    # Step 4: Filter out small areas and draw bounding boxes
    threshold = 100
    # Iterate through stats to filter out small areas and draw bounding boxes
    for i in range(1, num_labels): # num_labels : connected number of labels
        area = stats[i, cv2.CC_STAT_AREA] # stats[i, cv2.CC_STAT_AREA]is the number of pixels(area) of ith label(group)
        if area > threshold:
            x, y, w, h = stats[i, cv2.CC_STAT_LEFT], stats[i, cv2.CC_STAT_TOP], stats[i, cv2.CC_STAT_WIDTH], stats[i, cv2.CC_STAT_HEIGHT]
            cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box on color image

    # Display the result (comment out)
    #plt.imshow(cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)) # Convert BGR to RGB for proper display with matplotlib
    #plt.show()

    cv2.imshow("Frame",color_image)

    result_vid.write(color_image)

result_vid.release()

# mp4 on mac

OpenCV: FFMPEG: tag 0x34363268/'h264' is not supported with codec id 27 and format 'mov / QuickTime / MOV'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


file not found
file not found
file not found
file not found
file not found
file not found


In [3]:
# calculating the bounding box i/u ; later

In [1]:
# ai -> color image
# camera capture color image only
# depth image only

# ai -> bounding box on color image
# train -> depth image  only for
# fine tuning

# pytorch model zoo
# torchvision models - models and weights ~~ 사이트 들어가기 -> object detection model!! 사용하면 됨
# yolo pytorch hub : codes 있음
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# video
result_video = 'output_video.mov'

# Inference
results = model(result_video)

# Results
results.print()
results.show()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)

Using cache found in /Users/sojeonglee/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-5 Python-3.11.5 torch-2.2.1 CPU

Fusing layers... 
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


UnidentifiedImageError: cannot identify image file 'output_video.mov'

In [5]:
# single frame -> comparison ; back ground frame

In [ ]:
import cv2
import torch
from PIL import Image

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# video
result_video = 'output_video.mov'

# Open the video file
cap = cv2.VideoCapture(result_video)

# Frame counter
frame_count = 0

# Process each frame
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    if ret:
        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convert the frame to PIL Image
        pil_image = Image.fromarray(frame_rgb)
        
        # Inference with YOLOv5
        results = model(pil_image)

        # Print and show results
        results.print()
        results.show()
        
        frame_count += 1
    else:
        break

# Release the video capture object
cap.release()


In [5]:
import cv2
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Video settings
video_path = 'output_video.MOV'
output_path = 'output_video_yolo.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Process each frame
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    if ret:
        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Inference with YOLOv5
        results = model(frame_rgb)
        
        # Draw bounding boxes on the frame
        annotated_frame = results.render()[0]
        
        # Convert the frame back to BGR
        annotated_frame_bgr = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)
        
        # Write the annotated frame to the output video
        out.write(annotated_frame_bgr)
    else:
        break

# Release the video capture object and close the output file
cap.release()
out.release()


Using cache found in /Users/sojeonglee/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-5 Python-3.11.5 torch-2.2.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
